In [1]:
import numpy as np
from csv import reader
from collections import Counter   
from sklearn.preprocessing import MultiLabelBinarizer
import matplotlib.pyplot as plt
%matplotlib inline

def load_dataset(filename):
    with open(filename, 'r') as dest_f:
        data_iter = reader(dest_f, delimiter=' ', quotechar='"')
        data = [data for data in data_iter]
        data_array = np.asarray(data)
        data_array = data_array.astype(int)
    return data_array

def word_freq_list(train_data, frequency):
    cnt = Counter()
    for i in range(train_data.shape[0]):
        cnt[train_data[i,1]] += train_data[i, 2]
    if frequency == 'all':
        word_freq_list = cnt.most_common()
    else:
        word_freq_list = cnt.most_common(frequency)
    return word_freq_list

def Bernoulli_fit(train_file, label_file, f, alpha, beta):
    train_data = load_dataset(train_file)
    train_label = load_dataset(label_file)
    
    # one-hot encoder
    mlb = MultiLabelBinarizer()
    y = mlb.fit_transform(train_label)
    
    # calculate pi
    class_frequency = np.sum(y, axis=0)
    pi = class_frequency * 1. / len(train_label) 
   
    # get the vocab_list
    vocab_list = word_freq_list(train_data, f)

    # use the vacab_list to make X(matrix): row -> docIdx, col -> {0, 1} whether the word appears in the doc
    X = np.zeros((len(train_label), f))
    vocab_dict = dict(vocab_list)
    # list of the word_index
    vocab_idx = [a[0] for a in vocab_list]
    for row in range(train_data.shape[0]):
        word_idx = train_data[row, 1]
        if word_idx in vocab_dict:
            docID = train_data[row, 0]
            X[docID - 1, vocab_idx.index(word_idx)] = 1 

    # calculate theta
    vocab = f
    classes = len(pi)
    doc = len(train_label)
    theta = np.zeros((vocab, classes))
    theta_smoothing = np.zeros((vocab, classes))
    for k in range(classes):
        # MLE: 出现word i 且 属于j class的文本篇数 / 属于j class的文本篇数
        for j in range(vocab):
            cnt = 0
            for i in range(doc):
                if X[i, j] == 1 and y[i, k] == 1:
                    cnt += 1
            # MLE: 出现word i 且 属于j class的文本篇数 / 属于j class的文本篇数
            # theta[j, k] = cnt * 1. / class_frequency[k]
            theta_smoothing[j, k] = (cnt + 1) * 1./ (class_frequency[k] + 2)
            
            # MAP with smoothing
            # theta_smoothing[j, k] = (cnt + alpha - 1 + 1) * 1. / (class_frequency[k] + beta + alpha - 2 + 2)
        
    return theta, theta_smoothing, pi, vocab_list

def Bernoulli_predict(test_file, test_label_file, theta, theta_smoothing, pi, vocab_list):
    test = load_dataset(test_file)
    label = load_dataset(test_label_file)

    predict = list()
    max_prob = 0.0
    predict_class = -1
    classes = len(pi)

    # make X_test(matrix)
    X_test = np.zeros((len(label), len(vocab_list)))
    vocab_dict = dict(vocab_list)

    # list of the word_index
    vocab_idx = [a[0] for a in vocab_list]
    for row in range(test.shape[0]):
        word_idx = test[row, 1]
        if word_idx in vocab_dict:
            docID = test[row, 0]
            X_test[docID - 1, vocab_idx.index(word_idx)] = 1
    
    for i in range(X_test.shape[0]):
        # iterate every doc, calculate joint probablity of words, classes
        max_prob = 0.0
        predict_class = -1
        for c in range(classes):
            conditional_prob = 1.0
            for j in range(X_test.shape[1]):
                if X_test[i, j] == 0:
                    conditional_prob += np.log(1 - theta_smoothing[j, c])
                else:
                    conditional_prob += np.log(theta_smoothing[j, c])

            if c == 0 or conditional_prob + np.log(pi[c]) > max_prob:
                max_prob = conditional_prob + np.log(pi[c])
                predict_class = c + 1 #
        predict.append(predict_class)
    label = label[:, -1]
    acc = sum(predict == label) * 1. / len(predict)  
    
#         # make confusion matrix
    confusion_matrix = np.zeros((theta.shape[1], theta.shape[1]))
    for i in range(len(predict)):
        confusion_matrix[int(label[i])-1, int(predict[i])-1] += 1
    
    # row: label; col: predict
    total = np.sum(confusion_matrix)
    precision = []
    recall = []
    TP = 0
    for i in range(confusion_matrix.shape[0]):
        TP += confusion_matrix[i, i]
        curr_label = 0
        curr_pred = 0
        for j in range(confusion_matrix.shape[0]):
            curr_label += confusion_matrix[i, j]
            curr_pred += confusion_matrix[j, i]
        if curr_label != 0:
            precision.append(confusion_matrix[i, i] * 1. / curr_label)
        if curr_pred != 0:
            recall.append(confusion_matrix[i, i] * 1. / curr_pred)
    precision = np.array(precision)
    recall = np.array(recall)
    acc = TP * 1. / total
    return predict, acc, precision, recall

def Multinomial_fit(train_file, label_file, f, alpha=1, alpha_i=2):
    train_data = load_dataset(train_file)
    train_label = load_dataset(label_file)

    # one-hot encoder
    mlb = MultiLabelBinarizer()
    y = mlb.fit_transform(train_label)
    # calculate pi
    class_frequency = np.sum(y, axis=0)
    pi = class_frequency * 1. / len(train_label) 

    # get the vocab_list
    vocab_list = word_freq_list(train_data, f)

    # use the vacab_list to make X(matrix): row -> docIdx, col -> {0, 1} whether the word appears in the doc
    X = np.zeros((len(train_label), f))
    vocab_dict = dict(vocab_list)
    # list of the word_index
    vocab_idx = [a[0] for a in vocab_list]
    for row in range(train_data.shape[0]):
        word_idx = train_data[row, 1]
        if word_idx in vocab_dict:
            docID = train_data[row, 0]
            X[docID - 1, vocab_idx.index(word_idx)] = train_data[row, 2] 
    
    # calculate theta
    vocab = f
    classes = len(pi)
    doc = len(train_label)
    theta = np.zeros((vocab, classes))
    theta_smoothing = np.zeros((vocab, classes))
    
    for k in range(classes):
        # word i 在k class的频率 / k class的总词频
        for j in range(vocab):
            cnt = 0
            total_frequency = 0
            for i in range(doc):
                if y[i, k] == 1:
                    total_frequency += np.sum(X[i, :])
                    cnt += X[i, j]
            # MLE
            # theta[j, k] = cnt * 1. / total_frequency
            theta_smoothing[j, k] = (cnt + alpha) * 1. / (total_frequency + vocab) 
            
            # MAP
            #theta_smoothing[j, k] = (cnt + alpha_i - 1 + 1) * 1. / (total_frequency + vocab * alpha_i - vocab + vocab)
            #theta_smoothing[j, k] = (cnt + alpha_i) * 1. / (total_frequency + vocab * alpha_i)
    return theta, theta_smoothing, pi, vocab_list

def Multinomial_predict(test_file, test_label_file, theta, theta_smoothing, pi, vocab_list):
    test = load_dataset(test_file)
    label = load_dataset(test_label_file)

    predict = list()
    max_prob = 0.0
    predict_class = -1
    classes = len(pi)

    # make X_test(matrix)
    X_test = np.zeros((len(label), len(vocab_list)))
    vocab_dict = dict(vocab_list)

    # list of the word_index
    vocab_idx = [a[0] for a in vocab_list]
    for row in range(test.shape[0]):
        word_idx = test[row, 1]
        if word_idx in vocab_dict:
            docID = test[row, 0]
            X_test[docID - 1, vocab_idx.index(word_idx)] = test[row, 2] 

    for i in range(X_test.shape[0]):
    # iterate every doc, calculate joint probablity of words, classes
        max_prob = 0.0
        predict_class = -1
        for c in range(classes):
            conditional_prob = 0.0 
            for j in range(X_test.shape[1]): 
                ## using log likelihood to prevent underflow and increase efficiency 
                conditional_prob += np.log(theta_smoothing[j, c]) * X_test[i, j]

            if c == 0 or (conditional_prob + np.log(pi[c])) > max_prob:
                max_prob = conditional_prob + np.log(pi[c])

                predict_class = c + 1
        predict.append(predict_class)
    label = label[:, -1] 
    acc = sum(predict == label) * 1. / len(predict) 
   
    # make confusion matrix
    confusion_matrix = np.zeros((theta.shape[1], theta.shape[1]))
    for i in range(len(predict)):
        confusion_matrix[int(label[i])-1, int(predict[i])-1] += 1
    
    # row: label; col: predict
    total = np.sum(confusion_matrix)
    precision = []
    recall = []
    TP = 0
    for i in range(confusion_matrix.shape[0]):
        TP += confusion_matrix[i, i]
        curr_label = 0
        curr_pred = 0
        for j in range(confusion_matrix.shape[0]):
            curr_label += confusion_matrix[i, j]
            curr_pred += confusion_matrix[j, i]
        if curr_label != 0:
            precision.append(confusion_matrix[i, i] * 1. / curr_label)
        if curr_pred != 0:
            recall.append(confusion_matrix[i, i] * 1. / curr_pred)
    precision = np.array(precision)
    recall = np.array(recall)
    acc = TP * 1. / total
    return predict, acc, precision, recall

def evaluation(train_file, label_file, test_file, test_label_file, V):
    Bernoulli_acc = list()
    Multinomial_acc = list()
    Bernoulli_precision = list()
    Multinomial_precision = list()
    Bernoulli_recall = list()
    Multinomial_recall = list()
    
    for freq in V:
        print("|V| = {}".format(freq))
        print("Calculating Bernoulli...")
        theta, theta_smoothing, pi, vocab_list = Bernoulli_fit(train_file, label_file, freq, alpha=2, beta=2)
        predict_b, acc_b, precision_b, recall_b = Bernoulli_predict(test_file, test_label_file, theta, theta_smoothing, pi, vocab_list)
        print("Accuracy(%) = {}").format(acc_b * 100)
        print("Average precision across classes = {}").format(np.mean(precision_b))
        print("Average recall across classes = {}").format(np.mean(recall_b))
        
        Bernoulli_acc.append(acc_b*100)
        Bernoulli_precision.append(precision_b)
        Bernoulli_recall.append(recall_b) 
        
        print("Calculating Multinomial...")
        theta, theta_smoothing, pi, vocab_list = Multinomial_fit(train_file, label_file, freq, alpha=1, alpha_i=2)
        predict_m, acc_m, precision_m, recall_m = Multinomial_predict(test_file, test_label_file, theta, theta_smoothing, pi, vocab_list)
        print("Accuracy(%) = {}").format(acc_m * 100)
        print("Average precision across classes = {}").format(np.mean(precision_m))
        print("Average recall across classes = {}").format(np.mean(recall_m))
        
        Multinomial_acc.append(acc_m*100)
        Multinomial_precision.append(precision_m)
        Multinomial_recall.append(recall_m)
        
    return Bernoulli_acc, Multinomial_acc, Bernoulli_precision, Multinomial_precision, Bernoulli_recall, Multinomial_recall 

def plot(acc_b_list, acc_m_list, precision_b_list, precision_m_list, recall_b_list, recall_m_list, V, idx):
    plt.figure(1)
    plt.xlabel('Frequency')
    plt.ylabel('Accuracy((%))')
    plt.plot(V, acc_m_list, linestyle='-', marker='o', label="multivariate event")
    plt.plot(V, acc_b_list, linestyle='-', marker='o', label="multivariate Bernoulli")
    plt.legend(loc='best')
    plt.title('Accuracy Against Frequency')
    plt.grid()
                                                                                                                                                                 
    
    # plot precision
    fig, ax = plt.subplots()
    index = np.arange(len(precision_m_list[idx]))
    bar_width = 0.35
    opacity = 0.8

    rects1 = plt.bar(index, precision_b_list[idx], bar_width,
                     alpha=opacity,
                     color='g',
                     label='multivariate Bernoulli')
    
    rects2 = plt.bar(index + bar_width, precision_m_list[idx], bar_width,
             alpha=opacity,
             color='b',
             label='multivariate event')                                                         

    plt.xlabel('Class')
    plt.ylabel('Precision')
    plt.title('Precision over 20 classes')
    plt.xticks(index + bar_width, list(range(1,21)))
    plt.legend()

    plt.tight_layout()
    plt.show()
    
    # plot recall
    fig, ax = plt.subplots()
    index = np.arange(len(recall_m_list[idx]))
    bar_width = 0.35
    opacity = 0.8

    rects1 = plt.bar(index, recall_b_list[idx], bar_width,
                     alpha=opacity,
                     color='g',
                     label='multivariate Bernoulli')
    
    rects2 = plt.bar(index + bar_width, recall_m_list[idx], bar_width,
             alpha=opacity,
             color='b',
             label='multivariate event')                                                         

    plt.xlabel('Class')
    plt.ylabel('Recall')
    plt.title('Recall over 20 Classes')
    plt.xticks(index + bar_width, list(range(1,21)))
    plt.legend()

    plt.tight_layout()
    plt.show()
                                                              
def main():
    train_file = "20news-bydate/matlab/train.data"
    label_file = "20news-bydate/matlab/train.label"
    test_file = "20news-bydate/matlab/test.data"
    test_label_file = "20news-bydate/matlab/test.label"
    V = [100, 500, 1000, 2500, 5000, 7500, 10000, 12500, 25000, 50000, "all"]
    acc_b, acc_m, precision_b, precision_m, recall_b, recall_m = evaluation(train_file, label_file, test_file, test_label_file, V)
    
    plot(acc_b, acc_m, precision_b, precision_m, recall_b, recall_m, V, idx=0)
    
if __name__=="__main__":
    main()

|V| = 100
Calculating Bernoulli...
Accuracy(%) = 18.0146568954
Average precision across classes = 0.176948986341
Average recall across classes = 0.178478309933
Calculating Multinomial...
Accuracy(%) = 23.890739507
Average precision across classes = 0.235067020538
Average recall across classes = 0.231143396779
|V| = 500
Calculating Bernoulli...
Accuracy(%) = 36.4290473018
Average precision across classes = 0.358637122429
Average recall across classes = 0.422883329232
Calculating Multinomial...
Accuracy(%) = 48.5676215856
Average precision across classes = 0.479834001384
Average recall across classes = 0.500659381327
|V| = 1000
Calculating Bernoulli...
Accuracy(%) = 46.2758161226
Average precision across classes = 0.454990171489
Average recall across classes = 0.5221675074
Calculating Multinomial...
Accuracy(%) = 60.0532978015
Average precision across classes = 0.59241627597
Average recall across classes = 0.608545108605
|V| = 2500
Calculating Bernoulli...


KeyboardInterrupt: 